In [1]:
import sys

from psutil import virtual_memory

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Sat May  7 13:35:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.65       Driver Version: 511.65       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:2D:00.0  On |                  N/A |
| 26%   38C    P5    14W / 250W |   1988MiB /  8192MiB |     33%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
# install the latest version of kaggle_environments
!pip install kaggle_environments
!pip install gym
!pip install stable-baselines3

In [3]:
#import shutil

#shutil.rmtree('Kore_Bot', ignore_errors=True)
#!git clone -b RL-Main https://github.com/DerrianHarris/Kore_Bot.git

In [4]:
import os
#import sys
#from google.colab import drive

DRIVE_LOC = './'#os.path.join('drive','MyDrive','Kore_Bot')

#drive.mount('/content/drive')
#sys.path.append('/content/Kore_Bot')

In [5]:
import gym
from KoreGymEnv import KoreGymEnv
from KoreGymEnvHelper import transform_observation, transform_actions

from kaggle_environments import make
from kaggle_environments.envs.kore_fleets.kore_fleets import random_agent, attacker_agent, balanced_agent, miner_agent

from stable_baselines3.common.env_checker import check_env
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import VecCheckNan, VecNormalize, DummyVecEnv, VecMonitor
from stable_baselines3.common import results_plotter
from stable_baselines3.common.callbacks import CallbackList, EvalCallback, CheckpointCallback

from matplotlib import pyplot as plt

from tqdm.notebook import tqdm

import pandas as pd

from random import shuffle

from gym.envs.registration import register
import torch as th
import numpy as np

th.autograd.set_detect_anomaly(True)
np.seterr(all='raise')

MAX_EPISODE_STEPS = 400

# Example for the CartPole environment
register(
    # unique identifier for the env `name-version`
    id="Kore-v1",
    # path to the class for creating the env
    # Note: entry_point also accept a class as input (and not only a string)
    entry_point=KoreGymEnv,
    # Max number of steps per episode, using a `TimeLimitWrapper`
    max_episode_steps=MAX_EPISODE_STEPS,
)

env = gym.make('Kore-v1',opponent=random_agent)#KoreGymEnv(random_agent)
check_env(env)

C:\Users\sethe\Documents\Projects\Python\Kore_Bot\venv\lib\site-packages\stable_baselines3\common\env_checker.py:27: UserWarning: It seems that your observation  is an image but the `dtype` of your observation_space is not `np.uint8`. If your observation is not an image, we recommend you to flatten the observation to have only a 1D vector
  f"It seems that your observation {key} is an image but the `dtype` "
C:\Users\sethe\Documents\Projects\Python\Kore_Bot\venv\lib\site-packages\stable_baselines3\common\env_checker.py:35: UserWarning: It seems that your observation space  is an image but the upper and lower bounds are not in [0, 255]. Because the CNN policy normalize automatically the observation you may encounter issue if the values are not in that range.
  f"It seems that your observation space {key} is an image but the "
C:\Users\sethe\Documents\Projects\Python\Kore_Bot\venv\lib\site-packages\stable_baselines3\common\env_checker.py:48: UserWarning: The minimal resolution for an ima

In [6]:
LOG_DIR = os.path.join(DRIVE_LOC,'log')
MODEL_DIR = os.path.join(DRIVE_LOC,'input','model')
MODEL_FILE = 'best_model.zip'

best_model_path = os.path.join(MODEL_DIR, 'Best_Model')
inc_model_path = os.path.join(MODEL_DIR, 'Inc_Models')

best_model_file = os.path.join(best_model_path, MODEL_FILE)
log_file = os.path.join(LOG_DIR,'0.monitor.csv')

os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(best_model_path, exist_ok=True)
os.makedirs(inc_model_path, exist_ok=True)

class ProgressBar(BaseCallback):
    def __init__(self, verbose=0):
        super(ProgressBar, self).__init__(verbose)
        self.pbar = None

    def _on_training_start(self):
        self.pbar = tqdm(total=self.locals['total_timesteps'])

    def _on_rollout_start(self):
        self.pbar.refresh()

    def _on_step(self):
         self.pbar.update()

    def _on_training_end(self):
        self.pbar.close()
        self.pbar = None

def get_actions(model, obs, config, deterministic=False):
    x_obs = transform_observation(False, obs, config, 12)
    actions, state = model.predict(x_obs, deterministic=deterministic)
    next_actions = transform_actions(actions, obs, config)
    return next_actions

def run_test(model, opponent,deterministic=False):
    env = make('kore_fleets', debug=True)
    config = env.configuration

    game_agents = [None, opponent]
    shuffle(game_agents)

    print('Agents:', game_agents)

    trainer = env.train(game_agents)
    obs = trainer.reset()

    while not env.done:
        actions = get_actions(model, obs, config, deterministic=deterministic)
        obs, reward, done, info = trainer.step(actions)

    env.render(mode='ipython', width=640, height=480)


In [7]:
opponent = random_agent

NUM_GAMES = 1000

TIMESTEPS = NUM_GAMES * MAX_EPISODE_STEPS

N_CPU = 1#os.cpu_count()
print('CPU Cores =', N_CPU)

env = DummyVecEnv([lambda: env])
env = VecCheckNan(env)
env = VecNormalize(env)
env = VecMonitor(env, log_file)

try:
    model = PPO.load(best_model_file)
    model.set_env(env)
    print('Loaded model')
except:
    print('Making model')
    model = PPO(policy='MlpPolicy',
                env=env,
                verbose=0,
                n_steps=MAX_EPISODE_STEPS,
                batch_size=50,
                learning_rate=0.00025,
                policy_kwargs={'use_expln': True,'normalize_images':False})

CPU Cores = 1
Making model


In [8]:
progressbar = ProgressBar()

checkpoint_callback = CheckpointCallback(save_freq=40000, save_path=inc_model_path)
eval_callback = EvalCallback(env, best_model_save_path=best_model_path,
                             log_path=LOG_DIR, eval_freq=4000)
callback = CallbackList([progressbar, eval_callback,checkpoint_callback])

np.set_printoptions(threshold=sys.maxsize)
model = model.learn(total_timesteps=TIMESTEPS, callback=callback)



  0%|          | 0/400000 [00:00<?, ?it/s]

flight plan will be truncated: flight plan for 1 must be at most 1
flight plan will be truncated: flight plan for 1 must be at most 1
flight plan will be truncated: flight plan for 1 must be at most 1
flight plan will be truncated: flight plan for 1 must be at most 1
flight plan will be truncated: flight plan for 1 must be at most 1
flight plan will be truncated: flight plan for 1 must be at most 1
flight plan will be truncated: flight plan for 4 must be at most 3
flight plan will be truncated: flight plan for 4 must be at most 3
flight plan will be truncated: flight plan for 4 must be at most 3
flight plan will be truncated: flight plan for 4 must be at most 3
flight plan will be truncated: flight plan for 4 must be at most 3
flight plan will be truncated: flight plan for 4 must be at most 3
flight plan will be truncated: flight plan for 5 must be at most 4
flight plan will be truncated: flight plan for 5 must be at most 4
flight plan will be truncated: flight plan for 5 must be at mo

KeyboardInterrupt: 

In [ ]:
# plt.style.use(['seaborn-whitegrid'])

# results_plotter.plot_results([LOG_DIR], TIMESTEPS, results_plotter.X_TIMESTEPS, 'Kore Timesteps')
# results_plotter.plot_results([LOG_DIR], TIMESTEPS, results_plotter.X_EPISODES, 'Kore Episodes')

In [ ]:
log_files = [os.path.join(LOG_DIR, f'{i}.monitor.csv') for i in range(N_CPU)]

for i, log_file in enumerate(log_files):
  if os.path.isfile(log_file):
    df = pd.read_csv(log_file, skiprows=1)

    fig = plt.figure(figsize=(8, 2))
    plt.subplot(1, 2, 1, label=log_file)
    df['r'].rolling(window=TIMESTEPS//1000).mean().plot(title=f'Rewards {i}')

    plt.subplot(1, 2, 2, label=log_file)
    df['l'].rolling(window=TIMESTEPS//1000).mean().plot(title=f'Lengths {i}')

    plt.tight_layout()
    plt.show()

In [ ]:
run_test(model, opponent, deterministic=False)